In [1]:
import numpy as np
import pandas as pd

Linear regression generally have the form of $Y_{i} = \theta_{0} + \theta_{1} x_{1} + \theta_{2} x_{2} + ...$ <br>
There are several ways to find the coefficients of the regression: <br>
1. Linear Algebra: $\hat{\theta} = (X^{T}X)^{-1}X^{T}Y$ (When X is invertible) <br>
2. Gradient Descent: In this case, we need to write out the loss function and try to minimize the loss. <br>
$\hspace{30mm}$ $F(x)$ = Loss Function = SE = $ \sum^{n}_{i=1} (Y_{i} - \hat{Y_{i}})^{2}$ <br>

In [12]:
class Linear_Regression():
    def __init__(self, alpha = 1e-10 , num_iter = 10000, early_stop = 1e-50, intercept = True, init_weight = None):
        
        
        """
            Some initializations, if neccesary
            
            attributes: 
                        alpha: Learning Rate, default 1e-10
                        num_iter: Number of Iterations to update coefficient with training data
                        early_stop: Constant control early_stop.
                        intercept: Bool, If we are going to fit a intercept, default True.
                        init_weight: Matrix (n x 1), input init_weight for testing.
                        
            
            TODO: 1. Initialize all variables needed.
        """
        
        self.model_name = 'Linear Regression'
        
        self.alpha = 0
        self.num_iter = 100
        self.early_stop = 100
        self.intercept = 1
        self.init_weight = init_weight  ### For testing correctness.
        
    
    def fit(self, X_train, y_train):
        """
            Save the datasets in our model, and perform gradient descent.
            
            Parameter:
                X_train: Matrix or 2-D array. Input feature matrix.
                Y_train: Matrix or 2-D array. Input target value.
                
                
                TODO: 2. If we are going to fit the intercept, add a col with all 1's to the first column. (hint: np.hstack, np.ones)
                      3. Initilaize our coef with uniform from [-1, 1] with the number of col in training set.
                      4. Call the gradient_descent function to train.
        """
        
        self.X = np.mat(X_train)
        self.y = np.mat(y_train).T
        
        if self.intercept:
            ones = 0
            self.X = self.X
        
        self.coef = self.init_weight #### Please change this after you get the example right.
        
    def gradient(self):
        """
            Helper function to calculate the gradient respect to coefficient.
            
            TODO: 5. Think about the matrix format of the gradient of the loss function.
        """
        
        self.grad_coef = 0
        
    def gradient_descent(self):
        
        """
            Training function
            
            TODO: 6. Calculate the loss with current coefficients.
                  7. Update the temp_coef with learning rate and gradient.
                  8. Calculate the loss with temp_coef.
                  9. Implement the self adeptive learning rate. 
                      a. If current error is less than previous error, increase learning rate by a factor 1.3. 
                         And update coef, with temp_coef.
                      b. If previous error is less than current error, decrease learning rate by a factor of 0.9.
                         Don't update coef.
                  10. Add the loss to loss list we create.
        """
        
        self.loss = []
        
        for i in range(self.num_iter):

                
                
            self.gradient()

            
            previous_y_hat = 0
            
            
            temp_coef = 0
            
            ones = 0  # Matrix with 1's (1 x n), help with calculate the sum of a mattrix. hint: Think about dot product.
            
            pre_error = 0
            
            current_error = 0
            
            ### This is the early stop, don't modify fllowing three lines.
            if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
                self.coef = temp_coef
                return self
            
            if current_error <= pre_error:
                pass
            else:
                pass
                
            self.loss.append(0)
            
            if i % 10000 == 0:
                print('Iteration: ' +  str(i))
                print('Coef: '+ str(self.coef))
                print('Loss: ' + str(current_error))            
        return self
    
    def ind_predict(self, x: list):
        """
            Predict the value based on its feature vector x.

            Parameter:
            x: Matrix, array or list. Input feature point.
            
            Return:
                result: prediction of given data point
        """
        
        """
            TODO: 11. Implement the prediction function
        """
        result = 1
        
        return result
    
    def predict(self, X):
        """
            X is a matrix or 2-D numpy array, represnting testing instances. 
            Each testing instance is a feature vector. 
            
            Parameter:
            X: Matrix, array or list. Input feature point.
            
            Return:
                ret: prediction of given data matrix
        """
        
        """
            TODO: 12. Make sure add the 1's column like we did to add intercept.
                  13. Revise the following for-loop to call ind_predict to get predictions.

        """
        
        ret = []
        X = np.mat(X)
        if self.intercept:
            ones = 0
            X = X
        for x in X:
            ret.append(1)
        return ret
        
        

In [13]:
def min_max_normaliz(lst):
    """
    Helper function for normalize for faster training.
    """
    maximum = np.max(lst)
    minimum = np.min(lst)

    return (lst - minimum) / (maximum - minimum)

### We generate some easy data for testing. We should fit a line with, $Y = 30 * X + 20$

In [14]:
X = np.array(np.mat(np.arange(1, 1000, 5)).T)
y = np.array((30 * X)).flatten() + 20

#### Do NOT modify the following line, just run it when you are done.  You can also try different initialization, you will notice different coef at the end.

In [15]:
clf = Linear_Regression(alpha = 1, num_iter = 10000000, init_weight= np.mat([15,25]).T)
clf.fit(X,y)

/var/folders/lt/n_z12_lx2cb6n9qf36pmd3x40000gn/T/ipykernel_27706/2757621326.py:81: RuntimeWarning: invalid value encountered in double_scalars
  if (abs(pre_error - current_error) < self.early_stop) | (abs(abs(pre_error - current_error) / pre_error) < self.early_stop):
/var/folders/lt/n_z12_lx2cb6n9qf36pmd3x40000gn/T/ipykernel_27706/2757621326.py:78: RuntimeWarning: overflow encountered in power
  current_error = np.sum(np.power(y_pred - self.y, 2)) / self.X.shape[0]


Iteration: 0
Coef:      [[14.9999995 ]
 [24.99966767]]
Loss:      8321891.984726611
Alpha:     1.3000000000000002e-10
Iteration: 1000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.633170322859622e-22
Iteration: 2000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311577e+305
Alpha:     1.9484210902320901e-22
Iteration: 3000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.6092863265586054e-22
Iteration: 4000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.9199267675882357e-22
Iteration: 5000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.5857516173290704e-22
Iteration: 6000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.8918491548778715e-22
Iteration: 7000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.5625610870870621e-22
Iteration: 8000
Coef:      [[-2.47244678e+147

Iteration: 74000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.6559604767089905e-22
Iteration: 75000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311577e+305
Alpha:     1.975610426082858e-22
Iteration: 76000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.631743190032422e-22
Iteration: 77000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311577e+305
Alpha:     1.9467184780426773e-22
Iteration: 78000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.6078800645705816e-22
Iteration: 79000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.9182490549348105e-22
Iteration: 80000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.5843659209585117e-22
Iteration: 81000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.8901959776217947e-22
Iteration: 82000
Coe

Iteration: 145000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.703988317782028e-22
Iteration: 146000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311577e+305
Alpha:     2.032909078375999e-22
Iteration: 147000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.6790686568567444e-22
Iteration: 148000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311577e+305
Alpha:     2.0031791768289129e-22
Iteration: 149000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.6545134288880488e-22
Iteration: 150000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311577e+305
Alpha:     1.973884054709685e-22
Iteration: 151000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.6303173042937897e-22
Iteration: 152000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.945017353667268

Iteration: 212000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.8058412758000683e-22
Iteration: 213000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     2.154422706640687e-22
Iteration: 214000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.779432085192246e-22
Iteration: 215000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     2.1229157515876827e-22
Iteration: 216000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.7534091108914157e-22
Iteration: 217000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     2.0918695641517435e-22
Iteration: 218000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.7277667047488738e-22
Iteration: 219000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:  

...

Iteration: 2658000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.2513317781793706e-22
Iteration: 2659000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.0335297286134707e-22
Iteration: 2660000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.2330319088130014e-22
Iteration: 2661000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.0184150649010188e-22
Iteration: 2662000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.2149996624900648e-22
Iteration: 2663000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.0035214427830288e-22
Iteration: 2664000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.1972311254070327e-22
Iteration: 2665000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     9.888456296777275e-23
Iteration: 2666000
Coef:      [[-2.472446

Iteration: 2728000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.079289066071333e-22
Iteration: 2729000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.2876241683770994e-22
Iteration: 2730000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.0635052034202928e-22
Iteration: 2731000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.2687935476854613e-22
Iteration: 2732000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.0479521689394088e-22
Iteration: 2733000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.2502383119115226e-22
Iteration: 2734000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.0326265869249398e-22
Iteration: 2735000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.231954433739654e-22
Iter

Iteration: 2798000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.34463349012999e-22
Iteration: 2799000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.110591699477665e-22
Iteration: 2800000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.3249691471922357e-22
Iteration: 2801000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.0943500572735519e-22
Iteration: 2802000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.3055923817884438e-22
Iteration: 2803000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.0783459379517174e-22
Iteration: 2804000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.286498988294335e-22
Iteration: 2805000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Al

Iteration: 2866000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.1769753997427142e-22
Iteration: 2867000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.4041668890527425e-22
Iteration: 2868000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.159762941433647e-22
Iteration: 2869000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.3836319109707168e-22
Iteration: 2870000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.1428022035268065e-22
Iteration: 2871000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305
Alpha:     1.3633972428647484e-22
Iteration: 2872000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      inf
Alpha:     1.1260895047838942e-22
Iteration: 2873000
Coef:      [[-2.47244678e+147]
 [-1.64581755e+150]]
Loss:      8.988465674311578e+305

KeyboardInterrupt: 

####  As the number of iteration increase, you should notice the coeficient converges to [20, 30]. 
#### It maybe very slow update. Feel free to stop.

In [ ]:
clf.coef

In [ ]:
np.array(clf.predict(X))

#### Please try to normalize the X and fit again with normalized X. You should find something interesting. Also think about what you should do for predicting.

##### You can also try this with the wine dataset we use in HW1. Try fit this function to that dataset with same features. If you look closely to the updates of coefficients. What do you find? This could be mentioned in your report. 

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
url_Wine = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv'
wine = pd.read_csv(url_Wine, delimiter=';')
X = wine[['density','alcohol']]
y = wine.quality

In [ ]:
lr = LinearRegression()
lr.fit(X,y)
## Squared Error with sklearn.
sum((lr.predict(X) - y)**2)

#### You will notice different coefficients, but the loss is very close to each other like 805. In your report, briefly discuss this problem.

In [ ]:
clf = Linear_Regression(alpha = 1, num_iter = 5000000)
clf.fit(X,y)

In [ ]:
sum((clf.predict(X) - y)**2)